## Web Scraping for Rating Review Prediction Project 
### By: Ravinder Siingh

### Batch : Internship 28

## Web Scraping details:
### Data for this project is Scrap from Amazon & Flipkart.

In [45]:
# Web Scraping for Ratting Project Part - 1

In [73]:
#importing the required library
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [74]:
def get_template(search_term,page_no=1):
    """Generate a URL for search Term with page number"""
    search_term = search_term.replace(' ','+')
    template = f'https://www.amazon.in/s?k={search_term}&page={page_no}&qid=1623864210&ref=sr_pg_{page_no}'
    
    return template

In [75]:
def get_url():#the function that generated URL with the search tearm.
    iteams = ['laptops', 'Phones', 'Headphones', 'smart watches', 'Professional Cameras', 'Printers', 'scanners', 'keyboard', 'monitors', 'Home theater', 'router']
    URL = []     
    for i in iteams:
        for j in range(1,15):
            URL.append(get_template(i,j))
    return URL        

In [83]:
def get_review_url(): #the function that scraps the URL of the front page of the reviews and rattings with the search tearm.
    
    hreff = []
    driver = webdriver.Chrome(r"D:\chromedriver.exe") 
    driver.maximize_window()

    URL = get_url()
    for k in URL:
        driver.get(k) #Opening with the URL template

        for l in driver.find_elements_by_xpath("//a[@class = 'a-link-normal']"):
            hreff.append(l.get_attribute("href"))   
    return hreff

In [84]:
href = get_review_url()
len(href)

875

In [85]:
def get_review(): #the function that scraps the URL of all the reviews and rattings
    
    driver = webdriver.Chrome(r"D:\chromedriver.exe") #Calling the Web Driver
    driver.maximize_window()
    rattings = []
    review = []

    for i in href:
        driver.get(i) #Opening with the URL template
        
        try: #scraping the URL of the full review pages
            link = driver.find_element_by_xpath("//a[@data-hook='see-all-reviews-link-foot']")
            link = link.get_attribute("href")
            link= link.split('ref=cm')[0]
        except:
            pass     
        
        for i in range(1,5): #Scraping Reviews and Rattings
            l1= f'ref=cm_cr_getr_d_paging_btm_next_{i}?ie=UTF8&reviewerType=all_reviews&pageNumber={i}'
            l = link+ l1 
            
            
            driver.get(l)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            results = soup.find_all('div',{'class' : 'a-section celwidget'})
        
            for item in results:
                    try:
                        rev = item.find('span', {'data-hook': "review-body"})
                        atag = item.find(['a'], class_ = "a-link-normal")
                        if rev.span.text.replace('\n','').strip() and atag['title'][0]:
                            review.append(rev.span.text.replace('\n','').strip())
                            rattings.append(atag['title'][0])
                        else:
                            break
                    except:
                        break
       
    driver.close()
    
    rat = pd.DataFrame({'Rattings':rattings, "Review": review})
    
    #saving the dataframe in csv
    rat.to_csv("Rattings1.csv",index=False) #Creating CSV
    
    return rat


In [86]:
Rat = get_review()
Rat

,Rattings,Review
0,4,Colour is very unique. Performance is excellen...
1,4,impressive laptop
2,4,Product is superb goodI just love it there is...
3,5,You will never feel bad about any penny spend ...
4,3,Compare and buy. Good one but keyboard not wor...
...,...,...
24647,5,I have been using Google wifi first gen for al...
24648,4,Alright.. here is my view after using for 3 we...
24649,5,Netgear Orbi RBK50 model is among top notch me...
24650,5,It's been over a month now that I've been usin...


In [87]:
Rat.to_csv("Rattings_scraped.csv",index=False)# saving Data frame as CSV